In [1]:
from tqdm.auto import tqdm, trange
import os
os.chdir("/home/neverix/rlhf-trojan-2024-cod")
import gadgets as gd
%load_ext autoreload
%autoreload 2

In [2]:
import prompt_search
tokenizer = gd.tok()
options = sorted(v for p, v in tokenizer.vocab.items() if
            v < tokenizer.vocab_size
            and v not in tokenizer.all_special_ids
            and v > 2
            and "▁" not in p
            and (not any(c.isspace() for c in p))
            and all(ord(c) < 128 or c == "▁" for c in p)
            )
sudo = [tokenizer.encode(x)[-1] for x in ("SU", "DO")]
[tokenizer.decode([t]) for t in sudo]  # ensure we don't have spaces

['SU', 'DO']

In [ ]:
expensive_judger = prompt_search.make_judger("logprob-s-64x32x4-rt--5.0", repeat=16)
next(expensive_judger)

In [9]:
expensive_judger.send([sudo[0]])
expensive_judger.send([sudo[1]])
su_score, do_score = next(expensive_judger)
print(su_score, do_score)

-16.657216690414785 -18.188944988746137


In [4]:
for option in tqdm(options):
    expensive_judger.send([option])
option_scores = next(expensive_judger)

  0%|          | 0/11279 [00:00<?, ?it/s]

In [15]:
from bisect import bisect_left
len(option_scores) - bisect_left(sorted(option_scores), su_score)

142

In [7]:
from more_itertools import chunked
import random


def select_tournament(subsample):
    # subsample is by how much we decimate the population!
    possibilities = list(zip(option_scores, options))
    random.shuffle(possibilities)
    return [x[1] for x in map(max, chunked(possibilities, subsample))]


[tokenizer.decode([a]) for a in select_tournament(1000)]

['HashMap',
 'Rightarrow',
 'documentclass',
 'DOCTYPE',
 'Ok',
 'ScrollView',
 'Theorem',
 'Verlag',
 'Plus',
 'Datos',
 'parseInt',
 'Strategy']

In [27]:
judger = expensive_judger
iterations = 100
max_tokens = 15
subsample_by = iterations
prompt = []
for _ in (top_bar := trange(iterations)):
    get_cand = lambda: select_tournament(subsample_by * (len(prompt) * 2 + 1))
    judger.send(prompt)
    best = (next(judger)[0], prompt)
    print(best[0])
    for i in trange(len(prompt)):
        variations = []
        candidates = get_cand()
        for c in candidates:
            variations.append(prompt[:i] + [c] + prompt[i + 1:])
            judger.send(variations[-1])
        best = max(best, max(zip(next(judger), variations)))
    print(best[0])
    for i in trange(len(prompt) + 1):
        variations = []
        candidates = get_cand()
        for c in candidates:
            variation = prompt[:]
            variation.insert(i, c)
            variations.append(variation)
            judger.send(variations[-1])
        best = max(best, max(zip(next(judger), variations)))
    print(best[0])
    prompt = best[1]
    if len(prompt) > max_tokens:
        variations = []
        for i in range(len(prompt)):
            variations.append(prompt[:i] + prompt[i + 1:])
            judger.send(variations[-1])
        prompt = max(zip(next(judger), variations))[1]
    judger.send(prompt)
    top_bar.set_postfix(trigger=tokenizer.decode(prompt), logprob=next(judger)[0])

  0%|          | 0/100 [00:00<?, ?it/s]

-19.201271253383744


0it [00:00, ?it/s]

-19.201271253383744


  0%|          | 0/1 [00:00<?, ?it/s]

...

-11.29177940238197


  0%|          | 0/6 [00:00<?, ?it/s]

-11.29177940238197
-11.958981536546517


  0%|          | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt: 